In [4]:
import pandas as pd
import numpy as np
import time
import random
import pulp as pl

# =============================================================================
# 1. CONFIGURACIÓN (Modificada para 5 redes de 40 nodos y método exacto)
# =============================================================================
ARCHIVO_EXCEL = 'matriz_distancias_tsp.xlsx'
TAMANOS_REDES = [40]  # Modificado: Solo 40 nodos
CANTIDAD_POR_TAMANO = 5          # Modificado: 5 redes
SEED = 42                        # Para resultados reproducibles

# Los parámetros ACO han sido eliminados ya que se usará un método exacto (ILP)

random.seed(SEED)
np.random.seed(SEED)

# =============================================================================
# 2. PROCESAMIENTO DE DATOS (MÉTODO ROBUSTO) - Actualizado con manejo de errores
# =============================================================================
def cargar_y_normalizar_matriz(filepath):
    print(f"📂 Cargando archivo: {filepath} ...")
    try:
        df = pd.read_excel(filepath, header=None, index_col=None)
        # Convertir a numérico, forzando errores a NaN
        D = df.apply(pd.to_numeric, errors='coerce').values.astype(float)
    except Exception as e:
        print(f"❌ Error al cargar Excel: {e}")
        return None

    print(f"   Dimensión original: {D.shape}")

    # 1. Rellenar diagonales con 0
    np.fill_diagonal(D, 0)

    # 2. Hacer simétrica (Promedio de ida y vuelta para evitar inconsistencias)
    D = (D + D.T) / 2

    # 3. Manejo de NaNs (Asumir distancia muy grande si falta dato)
    max_val = np.nanmax(D) if not np.isnan(D).all() else 1000
    D = np.nan_to_num(D, nan=max_val)

    # 4. Limpieza de Outliers (Percentil 99)
    # Esto evita que un valor gigante (ej. 999999) arruine la escala
    perc99 = np.percentile(D, 99)
    D[D > perc99] = perc99

    # 5. Normalización a escala [0, 100]
    # Esto es CRUCIAL para que alpha y beta funcionen bien
    if D.max() != 0:
        D = (D / D.max()) * 100

    # 6. Evitar distancias 0 fuera de la diagonal (división por cero en heurística)
    # Reemplazamos 0s (no diagonal) por un valor pequeño (0.01)
    mask_zero = (D < 0.01) & (np.eye(D.shape[0]) == 0)
    D[mask_zero] = 0.01

    print("✅ Matriz maestra normalizada y limpiada (Escala 0-100).")
    return D

def generar_lotes_redes(matriz_maestra, sizes, n_per_size):
    """Genera diccionarios con submatrices listas para el algoritmo"""
    lote = []
    total_nodos = matriz_maestra.shape[0]

    for s in sizes:
        if s > total_nodos:
            print(f"⚠️ Saltando tamaño {s}: La matriz solo tiene {total_nodos} nodos.")
            continue

        for i in range(n_per_size):
            # Selección aleatoria de índices reales
            indices_reales = sorted(random.sample(range(total_nodos), s))

            # Extraer submatriz usando numpy slicing
            submatriz = matriz_maestra[np.ix_(indices_reales, indices_reales)]

            lote.append({
                "id_interno": f"{s}_nodos_v{i+1}",
                "n_nodos": s,
                "indices_reales": indices_reales, # Para saber qué nodos del Excel son
                "matriz_dist": submatriz
            })
    return lote

# =============================================================================
# 3. ALGORITMO EXACTO (ILP con PuLP)
# =============================================================================
# La función run_aco_tsp ha sido eliminada y su lógica reemplazada por ILP en la ejecución principal.

# =============================================================================
# 4. EJECUCIÓN PRINCIPAL
# =============================================================================

# A) Cargar Matriz
matriz_global = cargar_y_normalizar_matriz(ARCHIVO_EXCEL)

if matriz_global is not None:

    # B) Generar Redes
    print(f"\n⚙️ Generando redes de tamaños {TAMANOS_REDES}...")
    redes_a_procesar = generar_lotes_redes(matriz_global, TAMANOS_REDES, CANTIDAD_POR_TAMANO)
    print(f"   Total de redes generadas: {len(redes_a_procesar)}")

    resultados = []

    # C) Ejecutar Algoritmo ILP
    print(f"\n🚀 INICIANDO EJECUCIÓN ILP (método exacto con PuLP) para {len(redes_a_procesar)} redes\n")
    print(f"{'ID RED':<15} | {'NODOS':<6} | {'DISTANCIA':<10} | {'TIEMPO':<8}")
    print("-" * 50)

    for red in redes_a_procesar:
        num_cities_ilp = red['n_nodos']
        cities_labels = [f'C{i}' for i in range(num_cities_ilp)]
        dist_matrix_ilp = red['matriz_dist']

        distances_pulp = {}
        for i_idx, i_label in enumerate(cities_labels):
            for j_idx, j_label in enumerate(cities_labels):
                if i_idx != j_idx:
                    distances_pulp[(i_label, j_label)] = dist_matrix_ilp[i_idx, j_idx]

        model_ilp = pl.LpProblem(f"TSP_ILP_Demo_{red['id_interno']}", pl.LpMinimize)

        x_ilp = pl.LpVariable.dicts("x", [(i, j) for i in cities_labels for j in cities_labels if i != j], 0, 1, pl.LpBinary)

        model_ilp += pl.lpSum(distances_pulp[(i, j)] * x_ilp[(i, j)] for i in cities_labels for j in cities_labels if i != j), "Total_Distance"

        for i in cities_labels:
            model_ilp += pl.lpSum(x_ilp[(i, j)] for j in cities_labels if i != j) == 1, f"Out_Degree_{i}"
        for j in cities_labels:
            model_ilp += pl.lpSum(x_ilp[(i, j)] for i in cities_labels if i != j) == 1, f"In_Degree_{j}"

        u_ilp = pl.LpVariable.dicts("u", cities_labels, 1, num_cities_ilp, pl.LpInteger)
        M_ilp = num_cities_ilp
        start_node_ilp = cities_labels[0]
        for i in cities_labels:
            for j in cities_labels:
                if i != j and i != start_node_ilp and j != start_node_ilp:
                    model_ilp += u_ilp[i] - u_ilp[j] + M_ilp * x_ilp[(i, j)] <= M_ilp - 1, f"Subtour_Elimination_{i}_{j}"

        start_solve_time = time.time()
        model_ilp.solve()
        end_solve_time = time.time()
        tiempo_ilp = end_solve_time - start_solve_time

        if pl.LpStatus[model_ilp.status] == "Optimal":
            dist_ilp = pl.value(model_ilp.objective)
            tour_ilp_labels = []
            current_city_label = start_node_ilp
            while True:
                for next_city_label in cities_labels:
                    if current_city_label != next_city_label and x_ilp[(current_city_label, next_city_label)].varValue == 1.0:
                        tour_ilp_labels.append(current_city_label)
                        current_city_label = next_city_label
                        break
                if current_city_label == start_node_ilp:
                    tour_ilp_labels.append(start_node_ilp)
                    break

            tour_ilp_excel_indices = [red['indices_reales'][int(label[1:])] for label in tour_ilp_labels]

            print(f"{red['id_interno']:<15} | {red['n_nodos']:<6} | {dist_ilp:.2f}       | {tiempo_ilp:.2f}s")
            resultados.append({
                "Tamaño": red['n_nodos'],
                "Distancia": dist_ilp,
                "Tiempo": tiempo_ilp,
                "Tour_Excel": tour_ilp_excel_indices
            })
        else:
            print(f"{red['id_interno']:<15} | {red['n_nodos']:<6} | {'No óptimo':<10} | {tiempo_ilp:.2f}s")
            resultados.append({
                "Tamaño": red['n_nodos'],
                "Distancia": float('nan'),
                "Tiempo": tiempo_ilp,
                "Tour_Excel": []
            })

    # D) Resumen Final
    df_res = pd.DataFrame(resultados)
    print("\n" + "="*50)
    print("RESUMEN DE PROMEDIOS")
    print("="*50)
    if not df_res.empty:
        print(df_res.groupby("Tamaño")[["Distancia", "Tiempo"]].mean())
    else:
        print("No se generaron resultados.")

    print("\n✅ Proceso Finalizado.")

📂 Cargando archivo: matriz_distancias_tsp.xlsx ...
   Dimensión original: (290, 290)
✅ Matriz maestra normalizada y limpiada (Escala 0-100).

⚙️ Generando redes de tamaños [40]...
   Total de redes generadas: 5

🚀 INICIANDO EJECUCIÓN ILP (método exacto con PuLP) para 5 redes

ID RED          | NODOS  | DISTANCIA  | TIEMPO  
--------------------------------------------------
40_nodos_v1     | 40     | 1.73       | 4192.67s


KeyboardInterrupt: 